In [1]:
from bokvev.server import Server

server = Server(server_port=8081)
server.start()



In [2]:
from dataclasses import dataclass


@dataclass
class Video:
    data: str
    name: str


In [6]:
import numpy as np
import cv2
import tempfile
import base64
from edesdetectrl.dataloaders.echotiming import EchoTiming

echotiming = EchoTiming(None, 1)
video_index = 0


def get_video(video_index):
    with tempfile.NamedTemporaryFile("w+b", suffix=".mp4") as temp:
        task_fn, args = echotiming[video_index]
        data_item = task_fn(args)

        size = data_item.video.shape[1] + 8, data_item.video.shape[2]
        fps = 12
        out = cv2.VideoWriter(
            temp.name, cv2.VideoWriter_fourcc(*"mp4v"), fps, (size[1], size[0]), False
        )
        for i, frame in enumerate(data_item.video):
            progress = np.zeros((4, size[1]))
            progress[:, 0 : int((i * size[1] / data_item.length))] = 255
            data = np.concatenate(
                [
                    frame,
                    np.ones((4, size[1]))
                    * (50 if data_item.ground_truth[i] == 0 else 200),
                    progress,
                ],
                axis=0,
            )
            data = np.array(data, dtype="uint8")
            out.write(data)
        out.release()

        encoded = base64.b64encode(temp.read()).decode("utf-8")
    return {"video": Video(data=encoded, name=data_item.name)}


def next_video():
    global video_index
    video_index += 1
    return get_video(video_index)


def prev_video():
    global video_index
    video_index -= 1
    return get_video(video_index)


def index():
    with open("apps/video_visualizer/index.html", "r") as f:
        content = f.read()
    return content
